<h2>Blum-Goldwasser Probabilistic Encryption</h2>

In [12]:
import numpy 
import math 
import random
import time

In [83]:
#Umbral a partir del cual consideramos que un numero es "grande"
umbral = 2
#Rango de valores a partir del umbral que se pueden generar
rango = 2000

p, q, a, b = generarClavePrivada(umbral, rango)

clavePrivada = [p, q, a, b]
clavePublica = p*q

print("Clave privada --> ", clavePrivada)
print("Clave publica --> ", clavePublica)

msjClaro = leerFichero("msjClaro.txt")
alf = leerFichero("alf.txt")
msjCifrado, t = encriptar(msjClaro, alf, clavePublica)
print(msjCifrado)

#TODO esta comentado porque he hardcodeado la funcion desencriptar para testear
#msjDescifrado = desencriptar(p, q, a, b, t, msjCifrado)
#print(mensajeDescifrado)

Clave privada -->  [743, 883, -82, 69]
Clave publica -->  656069
[8, 1, 12, 12, 5, 0, 5, 12, 14, 3, 1, 5, 8, 2, 3, 12, 9, 5, 14, 4, 10, 7, 13, 12, 11, 7, 13, 1, 4, 3, 8, 0, 13, 13, 5, 7, 10, 1, 4, 13, 15, 12, 5, 9, 12, 3, 4, 11, 7, 6, 0, 11, 8, 6, 5, 5, 350681]


In [68]:
def generarClavePrivada(umbral, rango):
    p=0; q=0

    while not cumpleCongruencia(p, 3, 4) or not esPrimo(p):
        p = random.randint(umbral, umbral+rango)

    while not cumpleCongruencia(q, 3, 4)  or p==q or not esPrimo(q): 
        q = random.randint(umbral, umbral+rango)

    a, b = calcularAlgEuclidesExtendido(p, q)
    
    return p, q, a, b

In [69]:
def esPrimo(n):
    for i in range(2,int(n**0.5)+1):
        if n%i==0:
            return False

    return True

In [70]:
def cumpleCongruencia(num, numCongruente, modulo):
    return (num%modulo) == (numCongruente%modulo)

In [71]:
def calcularAlgEuclidesExtendido(p, q):
    restos = [p,q]
    listaA = [1,0]
    listaB = [0,1]
    cocientes = [0,0]

    i=2

    while(restos[i-1] > 1):
        cocientes.append(restos[i-2]//restos[i-1])
        restos.append(restos[i-2]%restos[i-1])
        listaA.append(listaA[i-2] - cocientes[i] * listaA[i-1])
        listaB.append(listaB[i-2] - cocientes[i] * listaB[i-1])
        i += 1

    if restos.pop()==0:
        return listaB.pop(), 0
    
    return listaA.pop(), listaB.pop()


In [72]:
def leerFichero(ruta):
    fichero = open(ruta, "r", encoding="utf8")
    texto = fichero.read()
    fichero.close()
    return texto

In [82]:
def encriptar(msjClaro, alf, n):

    k = math.floor(math.log(n,2))
    #h es la longitud de los bloques en los que se dividira el mensaje binario
    h = math.floor(math.log(k,2))
    x = []
    c = []

    msjBinario = strToBinary(msjClaro, alf)
    
    bloquesMsjBinario = separarBloques(msjBinario, h)
    t = len(bloquesMsjBinario)

    #print("k: ", k, " h: ", h)
    #print(bloquesMsjBinario)

    x.append(inicializarX0(n))

    for i in range(1, t+1):
        xi = pow(x[i-1], 2, n)
        #print("xi: ", xi)
        x.append(xi)

        p = decimalToBinary(x[i], 0)
        p = p[len(p)-h:]
        #print("p: ", p)

        ci = int(bloquesMsjBinario[i-1],2)^int(p,2)
        #print("ci: ", ci)
        c.append(ci)
    
    c.append(pow(x[len(x)-1], 2, n))

    return c, t

In [74]:
#def desencriptar(p, q, a ,b, t, msjCifrado):
def desencriptar():
    #OJO el mensaje cifrado se recibe como una lista de numeros
    p=499; q=547; a=-57; b=52; t=5; h=18; msjCifrado = [2,0,12,14,4,139680]

    d1 = pow((p+1)//4, t+1, p-1)
    d2 = pow((q+1)//4, t+1, q-1)
    lastX = msjCifrado.pop()
    u = pow(lastX, d1, p)
    v = pow(lastX, d2, q)
    n=p*q

    print(d1,d2,u,v)

    x = [(v*a*p+u*b*q)%(n)]
    m = []

    #HASTA AQUI ESTA BIEN

    for i in range(1, t+1):
        x.append(pow(x[i-1],2,n))
            
        p = decimalToBinary(x[i], 0)
        p = p[len(p)-h:]

        m.append(int(p,2)^msjCifrado[i-1])
    
    #AQUI HAY QUE HACER LA SIGUIENTE MOVIDA:
    #Usar x0 para construir xi y pi igual que al encriptar 
    #Obtener el mensaje original --> pi XOR ci

    return m


In [75]:
desencriptar()

463 337 20 24


[180537, 193932, 245601, 14, 4]

In [76]:
def strToBinary(cadena, alf):
    #numero minimo de digitos binarios para representar todas las letras del alfabeto
    longBinaria = math.ceil(math.log(len(alf),2)) 
    cadenaBinaria = ''

    for caracter in cadena:
        cadenaBinaria += decimalToBinary(alf.find(caracter), longBinaria)

    return cadenaBinaria



In [77]:
def decimalToBinary(caracterNumerico, longBinaria):
    bloqueBinario = bin(caracterNumerico)[2:]

    if(longBinaria==0):
        longBinaria = len(bloqueBinario)

    #Rellena con 0s a la izquierda 
    while(len(bloqueBinario)!=longBinaria):
        bloqueBinario = '0' + bloqueBinario
    
    return bloqueBinario


In [78]:
def inicializarX0(n):
    r = random.randint(1, n-1)
    while not tieneInverso(r, n):
        r = random.randint(1, n-1)
    
    return pow(r, 2, n)

In [79]:
def tieneInverso(numero, modulo):
    
    try:
        pow(numero,-1,modulo)
        return True
    except:
        return False
        
    return False

In [80]:
def separarBloques(texto, longitudBloque):
    temp = list(texto) #Copia
    bloques = []

    while(temp):
        bloque = ''
        for i in range(longitudBloque):
            if(len(temp)>0):
                bloque = temp.pop() + bloque
            else:
                bloque = '0' + bloque
        bloques.insert(0, bloque)

    return bloques